## Preprocessing

In [147]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [148]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME', 'ORGANIZATION'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,1,100000-499999,N,142590,1


In [149]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [150]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [151]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df[application_df['APPLICATION_TYPE'].map(application_df['APPLICATION_TYPE'].value_counts()) < 276]
application_types_to_replace = application_types_to_replace['APPLICATION_TYPE'].unique()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df = application_df.loc[application_df['APPLICATION_TYPE'] != 'T3']

In [152]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    4417
C3000    1888
C2000     885
C1300      51
C1200      12
C4000       6
C5000       2
C0          1
Name: CLASSIFICATION, dtype: int64

In [153]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df[application_df['CLASSIFICATION'].map(application_df['CLASSIFICATION'].value_counts()) > 1]['CLASSIFICATION'].value_counts()

C1000    4417
C3000    1888
C2000     885
C1300      51
C1200      12
C4000       6
C5000       2
Name: CLASSIFICATION, dtype: int64

In [154]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = application_df[application_df['CLASSIFICATION'].map(application_df['CLASSIFICATION'].value_counts()) < 1883]
classifications_to_replace = classifications_to_replace['CLASSIFICATION'].unique()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    4417
C3000    1888
Other     957
Name: CLASSIFICATION, dtype: int64

In [155]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
0,1,5000,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,5000,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
8,1,94389,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
9,1,5000,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
20,1,5000,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34291,1,5000,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
34292,1,5000,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
34294,1,5000,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
34295,1,5000,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [156]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL']).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [157]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [158]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 20)                680       
                                                                 
 dense_25 (Dense)            (None, 20)                420       
                                                                 
 dense_26 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1121 (4.38 KB)
Trainable params: 1121 (4.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [159]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [160]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100


171/171 [==============================] - 1s 1ms/step - loss: 0.6216 - accuracy: 0.6680
Epoch 2/100
171/171 [==============================] - 0s 1ms/step - loss: 0.5258 - accuracy: 0.7468
Epoch 3/100
171/171 [==============================] - 0s 1ms/step - loss: 0.5106 - accuracy: 0.7523
Epoch 4/100
171/171 [==============================] - 0s 1ms/step - loss: 0.5047 - accuracy: 0.7552
Epoch 5/100
171/171 [==============================] - 0s 1ms/step - loss: 0.5015 - accuracy: 0.7574
Epoch 6/100
171/171 [==============================] - 0s 1ms/step - loss: 0.4998 - accuracy: 0.7580
Epoch 7/100
171/171 [==============================] - 0s 1ms/step - loss: 0.4988 - accuracy: 0.7549
Epoch 8/100
171/171 [==============================] - 0s 1ms/step - loss: 0.4969 - accuracy: 0.7609
Epoch 9/100
171/171 [==============================] - 0s 1ms/step - loss: 0.4969 - accuracy: 0.7604
Epoch 10/100
171/171 [==============================] - 0s 1ms/step - loss: 0.4952 - accuracy: 0.7596
E

In [161]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

57/57 - 0s - loss: 0.4951 - accuracy: 0.7478 - 205ms/epoch - 4ms/step
Loss: 0.49505501985549927, Accuracy: 0.7477973699569702


In [162]:
nn.save('AlphabetSoupCharity_Optimization.h5')

/Users/villarroelvalentina/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
